In [9]:
# !pip install vncorenlp nbimporter
# !pip install pandas transformers torch underthesea vncorenlp tqdm hf_xet

data_process->word_segment ->̣̣̣̣(emb_different + complex) -> merged -> training-> check

In [10]:
import nbimporter
import importlib

from vncorenlp import VnCoreNLP
import json, re
import numpy as np
import pandas as pd
import torch
import math
from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD
#import mô hình
from underthesea import word_tokenize, pos_tag
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
#import file
from data_process import run_data_processed, clean_question, clean_word
from Embedding.word_segment import word_segment, ws_question, ori_question
from Embedding.emb_different import get_diff
from Embedding.sen_complex import run_sen_comlex

In [11]:
def data_process():
    subject_files = {
    "Văn_học": "output/văn.json",
    # "sử": "output/sử.json",
    # "địa": "output/địa.json",
    # "anh": "output/anh_văn.json"
    }
    all_data = {}
    for subject, file_path in subject_files.items():
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                all_data[subject] = json.load(f)
            print(f"Loaded {len(all_data[subject])} questions from {subject}")
        except FileNotFoundError:
            print(f"Warning: File not found - {file_path}")
            continue
    qa_pairs = []
    processed_qa = []
    clean_question(qa_pairs, all_data)
    clean_word(processed_qa, qa_pairs)
    run_data_processed(qa_pairs)

In [12]:
def run_embedding():
    input_file = r"data_processed\Văn_học_processed.txt"
    output_file = r"Embedding\Output_ws\qa_processed_ws.txt"
    print("---------Đang xử lý Word Segment--------------\n")
    word_segment(input_file, output_file)
    print("---------Đã xử lý xong Word Segment----------\n")

    #------------- xử lý data word segment--------------------------------
    print("---------Đang xử lý ws_question--------------\n")
    output_file_ws = r"Embedding\Output_ws\questions.json"
    ws_question(output_file, output_file_ws)
    print("---------Đã xử lý xong ws_question----------\n")

    #-------------clean data nguyên bản-----------------------
    print("---------Đang xử lý ori_question--------------\n")
    output_file_ori = r"Embedding\Output_ws\ori_questions.json"
    ori_question(input_file, output_file_ori)
    print("---------Đã xử lý xong ori_question----------\n")

In [13]:
def create_different_feartures():
    input = r"Embedding\Output_ws\questions.json"
    output = r"Embedding\Output_features"
    
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    model = AutoModel.from_pretrained("vinai/phobert-base")
    print(f"---------Đang tính toán độ lệch embedding giữa các options-------------\n")
    get_diff(input, output, tokenizer, model)


In [ ]:
def load_lit_terms(txt_file=r"create_lit_terms\lit_terms.txt"):
    """
    Đọc danh sách thuật ngữ từ file txt thành set
    """
    with open(txt_file, "r", encoding="utf-8") as f:
        terms = {line.strip().lower() for line in f if line.strip()}
    print(f"📖 Đã load {len(terms)} thuật ngữ từ {txt_file}")
    return terms

def create_sentence_comlex_feartures():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("⚡ Using device:", device)
    # PhoBERT cho embedding
    phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    phobert_model = AutoModel.from_pretrained("vinai/phobert-base").to(device)
    phobert_model.eval()
    # GPT2 tiếng Việt cho perplexity
    gpt2_tokenizer = AutoTokenizer.from_pretrained("NlpHUST/gpt2-vietnamese")
    gpt2_model = AutoModelForCausalLM.from_pretrained("NlpHUST/gpt2-vietnamese").to(device)
    gpt2_model.eval()

    connectors = {"và","hoặc","nhưng","tuy nhiên","cũng như là","nên","mà",
              "nếu","thì","bởi vì","vì","khi","mặc dù","để","sau","sau_khi",
              "trước","trước khi","hay","do","do đó","hễ","lẫn","cùng",
              "ngoài ra","vậy","trừ phi","hơn","bằng","như","dù cho",
              "nhằm","vì vậy","ngược lại","chỉ","chỉ trừ"}
    lit_terms = load_lit_terms("lit_terms.txt")
    svd = TruncatedSVD(n_components=128) 
    input_file = r"Embedding\Output_ws\ori_questions.json"
    output_file = r"Embedding\Output_features\question_features.csv"
    print("------------------------Đang tính toán độ phức tạp giữa các options------------------------")
    run_sen_comlex(input_file,output_file, connectors, phobert_tokenizer
                   , phobert_model, gpt2_tokenizer, gpt2_model, device, lit_terms, svd)


In [15]:
def merger_data():
    print(f"--------------------------------------Đang kết hợp data--------------------------------------")
    df1 = pd.read_csv(r"Embedding\Output_features\noise_features.csv")
    df2 = pd.read_csv(r"Embedding\Output_features\question_features.csv")
    df3 = pd.read_csv(r"training\data_for_training\van_with_bloom_out.csv")
    merged = pd.merge(df1, df2, on = "id", how = "inner")
    merged = pd.merge(merged, df3, on = "id", how = "inner")
    result = merged.drop(columns=["question_x", "answer", "subject", "question_y","question"])
    result.to_csv("training\merged.csv", index = False)
    print("Done.........")

In [16]:
def create_lit_terms():
    df1 = pd.read_csv(r"create_lit_terms\bold_words.csv", encoding="utf-8-sig")
    df2 = pd.read_csv(r"create_lit_terms\bold_words_2.csv", encoding = "utf-8-sig")

    df1["Bold Words"] = df1["Bold Words"].str.lower()
    df2["Bold Words"] = df2["Bold Words"].str.lower()

    merged = pd.concat([df1, df2], ignore_index=True)
    unique_words = merged.drop_duplicates(subset=["Bold Words"], keep="first")["Bold Words"]
    # Xuất TXT
    output_file = "lit_terms.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        for word in unique_words:
            f.write(word.strip() + "\n")

    print(f"✅ Đã tạo file {output_file}")
    print("🔢 Số lượng từ duy nhất:", len(unique_words))

In [17]:
# data_process()
run_embedding()

create_lit_terms()
create_different_feartures()
create_sentence_comlex_feartures()
merger_data()


---------Đang xử lý Word Segment--------------



Word Segmenting: 100%|██████████| 40013/40013 [07:21<00:00, 90.69it/s] 


✅ Word segmentation done.
---------Đã xử lý xong Word Segment----------

---------Đang xử lý ws_question--------------

✅ Đã chuyển 3097 câu hỏi sang JSON
---------Đã xử lý xong ws_question----------

---------Đang xử lý ori_question--------------

✅ Đã chuyển thành công 3097 câu hỏi sang file json
---------Đã xử lý xong ori_question----------

✅ Đã tạo file lit_terms.txt
🔢 Số lượng từ duy nhất: 213
---------Đang tính toán độ lệch embedding giữa các options-------------



  2%|▏         | 56/3097 [00:12<20:15,  2.50it/s]

Lỗi khi embedding câu hỏi id=59, môn=Văn_học
Lỗi khi embedding câu hỏi id=60, môn=Văn_học


...

Lỗi khi embedding câu hỏi id=2133, môn=Văn_học
Lỗi khi embedding câu hỏi id=2134, môn=Văn_học


100%|██████████| 3097/3097 [10:24<00:00,  4.96it/s]


--------Đã hoàn thành xong feartures tính độ lệch nhau giữa các options-------------
 Embedding\Output_features\noise_features.csv
⚡ Using device: cpu
📖 Đã load 212 thuật ngữ từ lit_terms.txt
------------------------Đang tính toán độ lệch giữa các options------------------------


Processing questions: 100%|██████████| 3097/3097 [06:58<00:00,  7.39it/s]


✅ Đã xuất độ phức tạp của câu hỏi sang file CSV: Embedding\Output_features\question_features.csv
   id  subject                                           question  \
0   1  Văn_học  Từ  Văn hóa trong văn bản  Phong cách Hồ Chí M...   
1   2  Văn_học  Ý nào nói lên việc tiếp thu có chọn lọc tinh h...   
2   3  Văn_học  Vấn đề chủ yếu được nói tới trong văn bản Phon...   
3   4  Văn_học  Theo tác giả quan niệm thẩm mĩ về cuộc sống củ...   
4   5  Văn_học         Từ nào sau đây trái nghĩa với truân chuyên   

   sentence_length  avg_word_length  num_clauses  num_punct  num_nouns  \
0                8         6.250000            0          0          4   
1               19         5.000000            0          0          7   
2               11         5.454545            0          0          4   
3               11         5.818182            0          0          5   
4                8         4.375000            1          0          2   

   num_verbs  num_adjs  ...   emb_118   emb